# **OSCN Scraper**

### Necessary Libraries

In [1]:
# from elasticsearch import Elasticsearch
# from elasticsearch_dsl import Search
from datetime import datetime
# from elasticsearch import helpers
import re
import numpy as np
import pandas as pd
import oscn
import json
import pickle
import zlib
from io import StringIO
from html.parser import HTMLParser
import csv


# es_client = Elasticsearch(http_compress=True)

### Search Parameters

In [2]:
# years = ['2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015','2016','2017','2018','2019','2020','2021']
years = ['2021']
counties = ['tulsa']
types =['CF']

### Pull cases and save as .oscn files - **WARNING LONG RUN TIME WARNING** - 

In [5]:
for county in counties:
  for year in years:
    for type in types:      
      cases=oscn.request.CaseList(type=type, year=year,county=county)
      all_cases = [c for c in cases]
      file_name = f'OSCN_Scraper/{type}.{county}.{year}.oscn'
      f = open(file_name, 'wb')
      pickle.dump(all_cases, f)
      f.close()
      print(f'{file_name}: {len(all_cases)}')

OSCN_Scraper/CF.tulsa.2021.oscn: 1133


### Combine OSCN files into one case file

In [ ]:
saved_cases = []

case_count =  0 

for county in counties:
  for year in years:
    for type in types:      
      file_name = f'data/{type}.{county}.{year}.oscn'
      new_cases = pickle.load(open(file_name, 'rb'))
      new_case_count = len(new_cases)
      case_count += new_case_count
      print(f'{file_name} added {new_case_count}')
      saved_cases += new_cases
        
print(f'counted case: {case_count} length saved {len(saved_cases)}')

### Add column headers

In [ ]:
columns = ['Filed','County','Type','Judge' ,'Cnumber', 'Docket','Source']
case_data = lambda c: [c.filed, c.county, c.type, c.judge, c.number, c.docket, c.source]
all_cases = pd.DataFrame([case_data(c) for c in saved_cases], columns = columns)

### Convert Filed Date to datetime and extract year/month

In [ ]:
all_cases['Filed'] = pd.to_datetime(all_cases['Filed'])
all_cases['Year'], all_cases['Month'] = all_cases['Filed'].dt.year, all_cases['Filed'].dt.month

### Get count of cases by year and/or month

In [ ]:
# table with totals by year
yearcount = all_cases[["Docket", "Year"]]
yearcount.rename(columns = {'Docket':'DocketCount'}, inplace = True) 
yearcount.groupby(['Year']).count()

# table with totals by year & month
#monthcount = all_cases[["Docket", "Year", "Month"]]
#monthcount.rename(columns = {'Docket':'DocketCount'}, inplace = True) 
#monthcount.groupby(['Year', 'Month']).count()

In [ ]:
pd.set_option('display.max_colwidth', 40) # Set to max (-1) to view the whole JSON string
df = pd.DataFrame(all_cases)
df['CaseNumber'] = "CF-" + df['Year'].astype(str) + "-" + df['Cnumber'].astype(str)
df

In [ ]:
# TESTING -------

In [ ]:
from pprint import pprint
pprint(df.to_dict('Docket'))

In [ ]:
from io import StringIO
from html.parser import HTMLParser

class MLStripper(HTMLParser):
    def __init__(self):
        super().__init__()
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.text = StringIO()
    def handle_data(self, d):
        self.text.write(d)
    def get_data(self):
        return self.text.getvalue()

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

In [ ]:
dock = strip_tags(df.Docket)